In [1]:
import torch
from torch import nn

In [2]:
def fn(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b

In [32]:
new_fn = torch.compile(fn, backend="inductor", 
                       options={'trace.graph_diagram':True,
                                'trace.enabled':True})

input_tensor = torch.randn(10000, requires_grad=True)
out = new_fn(input_tensor, input_tensor).sum().backward()

Writing FX graph to file: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_18_07_19_15_866950-pid_184/aot_torchinductor/model__0_forward_1.0/graph_diagram.svg


[2023-03-18 07:19:21,225] torch._inductor.debug: [WARNING] model__0_forward_1 debug trace: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_18_07_19_15_866950-pid_184/aot_torchinductor/model__0_forward_1.0


Writing FX graph to file: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_18_07_19_15_866950-pid_184/aot_torchinductor/model__0_backward_2.1/graph_diagram.svg


[2023-03-18 07:19:22,767] torch._inductor.debug: [WARNING] model__0_backward_2 debug trace: /pytorch-examples/pytorch-graph-optim/torch_compile_debug/run_2023_03_18_07_19_15_866950-pid_184/aot_torchinductor/model__0_backward_2.1


In [34]:
from torch.fx import passes, symbolic_trace
model = symbolic_trace(fn)

g = passes.graph_drawer.FxGraphDrawer(model, 'fn')
# with open("unoptimized_graph.svg", "wb") as f:
#     f.write(g.get_dot_graph().create_svg())

In [31]:
from functorch import make_fx
input_tensor = torch.randn(10000, requires_grad=True)
g = make_fx(fn)(input_tensor, input_tensor)
print(g.code)




def forward(self, x_1, y_1):
    sin = torch.ops.aten.sin.default(y_1)
    cos = torch.ops.aten.cos.default(y_1);  y_1 = None
    add = torch.ops.aten.add.Tensor(sin, cos);  sin = cos = None
    return add
    


In [37]:
print(model.code)




def forward(self, x, y):
    sin = torch.sin(x);  x = None
    cos = torch.cos(y);  y = None
    add = sin + cos;  sin = cos = None
    return add
    


In [ ]:
from functorch import make_fx
g = make_fx(model)(x)